# Import empirical data

In [129]:
import functools
import itertools
import json

import pandas as pd

from collections.abc import Mapping


d = pd.read_json('../../../model/lib_learning_output/synthesis_output_cogsci_revised/ca_synthesis_cogsci_21_ppt_1.json')
d['dsl'][0]

['h',
 'v',
 'l_0',
 'l_1',
 'l_2',
 'l_3',
 'l_4',
 'l_5',
 'l_6',
 'l_7',
 'l_8',
 'l_9',
 'l_10',
 'l_11',
 'l_12',
 'r_0',
 'r_1',
 'r_2',
 'r_3',
 'r_4',
 'r_5',
 'r_6',
 'r_7',
 'r_8',
 'r_9',
 'r_10',
 'r_11',
 'r_12']

## Constructing the lexicon

In [87]:
# we assume all agents start with a basic mapping 
# between 'h'/'v' in the DSL and 'horizontal'/'vertical' in language
class BlockLexicon(dict) :
    def __init__(self, primitives, lexemes):
        """
        initialize dictionary subclass
        """
        dict.__init__(self)
        self.__dict__ = self
        unassigned_lexemes = lexemes.copy()
        
        for primitive in primitives :
            if primitive in ['v', 'h'] :
                adjective = 'horizontal' if primitive == 'h' else 'vertical'
                self.update({primitive : f'place a {adjective} block.'})
            elif primitive[0] in ['l', 'r'] :
                distance = primitive.split('_')[1]
                direction = 'right' if primitive[0] == 'r' else 'left'
                self.update({primitive : f'move to the {direction} by {distance}'})
            else :
                self.update({primitive: f'place a {unassigned_lexemes.pop()}.'})
    def __hash__(self):
        return hash(json.dumps(self, sort_keys=True))

    def invert(self):
        """
        invert keys and values of a dictionary d
        """
        return {v: k for k, v in self.items()}
    
    def dsl_to_language(self, e) :
        # parse expression e written in DSL into language
        return self.get(e)
    
    def language_to_dsl(self, e) :
        # parse expression e written in DSL into language
        return self.invert().get(e)


Let's take this class out for a drive. 

We initialize it with the primitives of the agent's DSL on a given trial and an (ordered) list of available lexemes.

In [88]:
alpha = 0.04
lexemes = ['blah', 'blab', 'bloop', 'bleep']
dsl = d['dsl'][10]
l = BlockLexicon(dsl, lexemes)
print(dsl[0], '->', l.dsl_to_language(dsl[0]))
print(dsl[10], '->', l.dsl_to_language(dsl[10]))
print(dsl[-1], '->', l.dsl_to_language(dsl[-1]))

h -> place a horizontal block.
l_8 -> move to the left by 8
chunk_C -> place a blah.


and we can also go in the other direction

In [89]:
print('place a horizontal block. ->', l.language_to_dsl('place a horizontal block.'))
print('move to the left by 8 ->', l.language_to_dsl('move to the left by 8'))
print('place a blah. ->', l.language_to_dsl('place a blah.'))

place a horizontal block. -> h
move to the left by 8 -> l_8
place a blah. -> chunk_C


## Adding probabilities

If we were using a probabilistic programming language like WebPPL, we would be able to automatically construct probability distributions over lexicons. But to do this simple example in base python, we're going to manually construct a distribution as another dictionary. The keys will be possible lexicons and the values will be their probabilities. 

Because the only thing that varies across different lexicons in our example is the word to use for a given chunk, the support of the distribution only needs to be defined over the list of possible mappings (everything else is fixed across lexicons)

In [195]:
class Distribution(dict) :
    def __init__(self, support, probabilities):
        super().__init__(self)
        self.__dict__ = self
        for element, probability in zip(support, probabilities) :
            self.update({element: probability})

    def update(self, element):
        for k, prob in element.items():
            if k in self :
                # if it already exists in the distribution, aggregate probabilities
                self[k] += prob
            else : 
               # otherwise add as a new element of the distribution
                self[k] = prob

class UniformDistribution(Distribution) :
    def __init__(self, support):
        uniform_probabilities = [ 1/len(support) ] * len(support)
        super().__init__(support, uniform_probabilities)
        
class MarginalDistribution(Distribution) :
    def __init__(self, support):
        uniform_probabilities = [ float(0) ] * len(support)
        super().__init__(support, uniform_probabilities)

We can now define a prior over lexicons:

In [196]:
possible_lexicons = [BlockLexicon(dsl, list(mapping)) for mapping in itertools.permutations(lexemes)]
prior = UniformDistribution(possible_lexicons)
list(prior.items())[:1]

[({'h': 'place a horizontal block.',
   'v': 'place a vertical block.',
   'l_0': 'move to the left by 0',
   'l_1': 'move to the left by 1',
   'l_2': 'move to the left by 2',
   'l_3': 'move to the left by 3',
   'l_4': 'move to the left by 4',
   'l_5': 'move to the left by 5',
   'l_6': 'move to the left by 6',
   'l_7': 'move to the left by 7',
   'l_8': 'move to the left by 8',
   'l_9': 'move to the left by 9',
   'l_10': 'move to the left by 10',
   'l_11': 'move to the left by 11',
   'l_12': 'move to the left by 12',
   'r_0': 'move to the right by 0',
   'r_1': 'move to the right by 1',
   'r_2': 'move to the right by 2',
   'r_3': 'move to the right by 3',
   'r_4': 'move to the right by 4',
   'r_5': 'move to the right by 5',
   'r_6': 'move to the right by 6',
   'r_7': 'move to the right by 7',
   'r_8': 'move to the right by 8',
   'r_9': 'move to the right by 9',
   'r_10': 'move to the right by 10',
   'r_11': 'move to the right by 11',
   'r_12': 'move to the right b

# Create agents

Now we're ready to define our Architect and Builder.

In [214]:
import numpy.random as random

class Agent() :
    def __init__(self, role, trial) :
        self.role = role
        self.dsl = trial['dsl']
        self.beliefs = LexiconDistribution(self.dsl, possible_mappings, uniform_probabilities)
        self.utterances = [*[*self.beliefs.keys()][0].values()]

    
    def build(self, utt) :
        ''' 
        get distribution over dsl actions
        marginalizing over different possible meanings
        '''
        action_dist = MarginalDistribution(self.dsl)
        for lexicon, prob in self.beliefs.items() :
            action_dist.update({lexicon.language_to_dsl(utt) : prob})
        return random.choice(a = [*action_dist.keys()], 
                             p = [*action_dist.values()])
        
    def speak(self, target) :
        '''
        construct distribution over utterances
        marginalizing over different possible meanings
        '''
        utt_dist = MarginalDistribution(self.utterances)
        for lexicon, prob in self.beliefs.items() :
            utt_dist.update({lexicon.dsl_to_language(target) : prob})
        return random.choice(a = [*utt_dist.keys()], 
                             p = [*utt_dist.values()])


In [217]:
architect = Agent('architect', d.loc[0].to_dict())
print('architect choice: ', architect.speak('h'))

builder = Agent('builder', d.loc[0].to_dict())
print('builder choice: ', builder.build('place a horizontal block.'))

architect choice:  place a horizontal block.
builder choice:  h


# Run simulation

Now we have our agents, we just have to run them forward

In [224]:
for i, trial in d.iterrows() :
  # need to sample target program based on program length. Use currTrial.
    possiblePrograms = list(trial['programs_with_length'].keys())
    print(possiblePrograms)
    lengths = np.array(trial['programs_with_length'].values()) 
    utilities = np.exp(-alpha * lengths)
#   var target = sample(Categorical({vs: possiblePrograms, ps: lengths}));
#   var numInstructions = currTrial.programs_with_length[target];
#   var newData = reduce(function(targetInstruction, acc) {
#     var speakerDist = Architect(targetInstruction, currTrial, prevData);
#     var utt = sample(speakerDist);
#     var listenerDist = Builder(utt, currTrial, prevData);
#     var response = sample(listenerDist);
#     return {utt: acc.utt.concat(utt),
#             response: acc.response.concat(response),
#             targetInstruction: acc.targetInstruction.concat(targetInstruction),
#             numCorrect: acc.numCorrect + (response == targetInstruction)};
#   }, {
#     utt: [], response: [], targetInstruction: [], numCorrect: 0
#   }, target.split(' '));
#   return step(rest(remainingTrials),
#               prevData.concat(extend(newData, {numInstructions})));
# };

['h l_4 h l_1 v v r_9 v r_6 v l_5 h r_4 h']
['h l_4 h l_1 v v r_12 h l_1 v v r_1 h']
['h l_1 v v r_1 h r_6 v r_6 v l_5 h r_4 h']
['v r_6 v l_5 h r_4 h r_7 chunk_8b r_1 h', 'v r_6 v l_5 h r_4 h r_7 h l_1 v v r_1 h']
['chunk_8b r_1 h r_12 h l_4 chunk_8b', 'h l_1 v v r_1 h r_12 h l_4 h l_1 v v']
['chunk_Pi r_9 chunk_L', 'v r_6 v l_5 h r_4 h r_9 chunk_L', 'chunk_Pi r_9 h l_4 h l_1 v v', 'v r_6 v l_5 h r_4 h r_9 h l_4 h l_1 v v']
['chunk_Pi r_9 chunk_L', 'v r_6 v l_5 h r_4 h r_9 chunk_L', 'chunk_Pi r_9 h l_4 h l_1 v v', 'v r_6 v l_5 h r_4 h r_9 h l_4 h l_1 v v']
['chunk_Pi r_9 chunk_L', 'v r_6 v l_5 h r_4 h r_9 chunk_L', 'chunk_Pi r_9 h l_4 h l_1 v v', 'v r_6 v l_5 h r_4 h r_9 h l_4 h l_1 v v']
['chunk_Pi r_9 chunk_L', 'v r_6 v l_5 h r_4 h r_9 chunk_L', 'chunk_Pi r_9 h l_4 h l_1 v v', 'v r_6 v l_5 h r_4 h r_9 h l_4 h l_1 v v']
['chunk_Pi r_9 chunk_L', 'v r_6 v l_5 h r_4 h r_9 chunk_L', 'chunk_Pi r_9 h l_4 h l_1 v v', 'v r_6 v l_5 h r_4 h r_9 h l_4 h l_1 v v']
['chunk_Pi r_9 chunk_L', 'v r_6